In [25]:
import numpy as np
import matplotlib.pyplot as plt
from pygsp import graphs, filters, plotting
import pygsp as pg
from numpy.random import default_rng
from numpy.random import randint
from random import sample
import random
import cvxpy as cp

plotting.BACKEND = 'matplotlib'
plt.rcParams['figure.figsize'] = (10, 5)

In [26]:
def createGraph(Nodes,i,graphType):
    while True:
        s = np.array(sample(range(1, 10000), 1))
        G = pg.graphs.Sensor(N=Nodes, seed = s)
        typeName = 'Sensor'
            
        if (G.is_connected() and not(G.is_directed())):
            break
    if i==0:        
        print(typeName)
           
    G.compute_fourier_basis()
    W = np.random.uniform(size=(2,1))
    x =  G.U[:,1:3]@W
    x = x/np.max(np.abs(x))
    eigenvalue = G.e
    print(eigenvalue[1:4])

    return G,x

In [27]:
def computeMainSignal(signals,k,N):
    x = np.zeros((N,1))
    for i in range(k):
        x = x + signals[:,i].reshape(N,1)
    return x

In [28]:
def computeSmoothness(signals,graphs,k):
    smoothness = []
    sumOverSmoothness = 0
    for i in range(k):
        smoothness.append(signals[:,i].T@graphs[i].L@signals[:,i])
        sumOverSmoothness = sumOverSmoothness + smoothness[i]
    return sumOverSmoothness,smoothness

In [29]:
def decomposeGraphSignal(mainSignal,signals,graphs,k,N):
    A = np.tile(np.identity(N), (1, k))
    
    H = np.zeros((N*k,N*k))
    for i in range(k):
        g = graphs[i].L
        H[i*N:(i+1)*N,i*N:(i+1)*N] = g.toarray()
        
        
    X = cp.Variable((N*k,1))
    objective = cp.Minimize(cp.quad_form(X, H)) 
    
    constraints = [ A @ X == mainSignal]
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=True)
    
    temp = X.value
    X_hat = temp.reshape(N,k, order='F')
    
    return X_hat

In [30]:
def makeMeanZero(signals,k):
    for i in range(k):
            signals[:,i] = signals[:,i] - np.mean(signals[:,i])
            
    return signals

In [31]:
def calculateSNR(est_signals,signals,k):
    SNR = np.zeros((1,k))
    for i in range(k):
        SNR[0,i]=(10*np.log10(np.linalg.norm(signals[:,i])**2/np.linalg.norm(signals[:,i]-est_signals[:,i])**2))
    return SNR

In [32]:
N = 100  # number of nodes
k = 5    # number of graphs
np.random.seed(0)
random.seed(0)
iteration = 200
av_snr = np.zeros((1,k))

for itera in range(1,iteration+1):
    print('############################################# iteration = ',itera,' ########################################')
    # create graph and graph signals
    graphType = randint(1, 3, 1)
    graph_list = []
    signals = np.zeros((N,k))
    for i in range(k):
        G , x = createGraph(N,i,graphType)
        graph_list.append(G)
        signals[:,[i]] = x

    # calculate the main signal : x = x1+x2+...+x4
    mainSignal = computeMainSignal(signals,k,N)
    mainSignal = mainSignal - np.mean(mainSignal)

    # decompose graph signal
    estimated_signals = decomposeGraphSignal(mainSignal,signals,graph_list,k,N)

    # calculate graph signals' smoothness
    sumOverSmoothness,smoothness = computeSmoothness(signals,graph_list,k)
    print('smoothness:')
    print(smoothness)
    print('sum over smoothness:',sumOverSmoothness)

    # calculate estimated graph signals' smoothness
    sumOverSmoothness_est,smoothness_est = computeSmoothness(estimated_signals,graph_list,k)
    print('smoothness_est:')
    print(smoothness_est)
    print('sum over smoothness_est:',sumOverSmoothness_est)

    # make mean of the estimated signals zero to remove DC values
    estimated_signals = makeMeanZero(estimated_signals,k)

    # calculate SNRs
    SNRs = calculateSNR(estimated_signals,signals,k)
    print('SNR:',SNRs)
    av_snr = av_snr + SNRs/iteration

    # compute objective values
    objectiveValue = sumOverSmoothness
    objectiveValue_est = sumOverSmoothness_est
    print('objectiveValue:',objectiveValue)
    print('objectiveValue_est:',objectiveValue_est)

    # compute error
    error = np.linalg.norm(mainSignal-estimated_signals@np.ones((k,1)))
    print('error:',error)
    
    
print('av_snr: ',av_snr)
print('total_av_snr: ',np.mean(av_snr))

############################################# iteration =  1  ########################################
Sensor
[0.37258419 0.4387519  0.66748598]
[0.05875928 0.28807215 0.32534728]
[0.11825027 0.18386095 0.34892946]
[0.13188762 0.24836801 0.32811761]
[0.22832208 0.31831286 0.57057818]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 500, constraints m = 100
          nnz(P) + nnz(A) = 3568
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: of

[0.21710875 0.2775315  0.38441583]
[0.21125473 0.23768327 0.39527917]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 500, constraints m = 100
          nnz(P) + nnz(A) = 3704
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   2.27e+00   3.03e+03   1.00e-01   8.07e-03s
  50   1.9720e+01   3.84e-09   4.08e-

[0.17767475 0.42033836 0.49380116]
[0.3216424  0.36576084 0.54743327]
[0.19534231 0.27252408 0.36441923]
[0.18032488 0.24005892 0.47943504]
[0.28325166 0.38601409 0.48471403]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 500, constraints m = 100
          nnz(P) + nnz(A) = 3720
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        

[0.16916663 0.24865815 0.44653883]
[0.2407239  0.25219457 0.59216968]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 500, constraints m = 100
          nnz(P) + nnz(A) = 3509
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   2.29e+00   3.03e+03   1.00e-01   7.38e-03s
  50   1.9890e+01   3.77e-09   4.08e-

Sensor
[0.10936596 0.15820464 0.19905809]
[0.14476067 0.18289815 0.23216035]
[0.15510059 0.26714613 0.49704745]


...

[0.16554908 0.24453181 0.33408959]
[0.17116059 0.243139   0.48659247]
[0.16393275 0.32794946 0.42412562]
[0.23983683 0.32176245 0.5513118 ]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 500, constraints m = 100
          nnz(P) + nnz(A) = 3612
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   2.23e+00  

[0.08553567 0.14327878 0.33679923]
[0.20965723 0.28144736 0.4496008 ]
[0.21323532 0.29123134 0.46634378]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 500, constraints m = 100
          nnz(P) + nnz(A) = 3560
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   2.31e+00   2.93e+03   1.00e-01   7.25e-03s
  

[0.12302519 0.37680814 0.56371355]
[0.20817992 0.28380218 0.40601241]
[0.22531104 0.26503081 0.48276238]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 500, constraints m = 100
          nnz(P) + nnz(A) = 3604
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   2.28e+00   1.54e+03   1.00e-01   6.86e-03s
  

          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   2.81e+00   2.83e+03   1.00e-01   7.27e-03s
  50   2.1916e+01   1.36e-08   3.82e-08   1.98e-02   2.12e-02s
plsh   2.1916e+01   3.52e-16   7.61e-15   --------   2.88e-02s

status:               solved
solution polish:      successful
number of iterations: 50
optimal objective:    21.9161
run time:             2.88e-02s
optimal rho estimate: 9.28e-03

smoothness:
[8.409457630996432, 6.511550317040351, 3.951850183311654, 0.8050951891269271, 5.842389172298491]
sum over smoothness: 25.520342492773857
smoothness_est:
[7.337984726441538, 4.771370347834459, 3.5595334627245356, 1.1807357257005433, 5.066521961662222]
sum over smoothness_est: 21.916146224363295
SNR: [[18.37082418 13.26646061 15.41900697 12.03729344 17.24966284]]
objectiveValue: 25.520342492773857
objectiveValue_est: 21.916146224363295
error: 2.2688034453

Sensor
[0.09326582 0.28840486 0.35656534]
[0.27769451 0.29963135 0.42981393]
[0.15199894 0.32492818 0.49439345]
[0.12168604 0.17221502 0.44932357]
[0.26639717 0.32762775 0.50085437]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 500, constraints m = 100
          nnz(P) + nnz(A) = 3482
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho 